In [1]:
from sklearn.decomposition import KernelPCA
from sklearn.cross_validation import KFold
from sklearn import svm
import numpy as np
import math
import random
import math
from sklearn.metrics import confusion_matrix as CM

##4*n features 

In [2]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_digits

In [3]:
def IndOfSortedFea(feaInfo):
    """return sorted index of a list"""
    sortInfo=sorted(feaInfo,reverse=True)
    ##for test
    ##print sortInfo
    return [feaInfo.index(i) for i in sortInfo]
#a=[3,4,6,2]
#print IndOfSortedFea(a)

In [4]:
def corDis(x,y):
    """
    calculate the correlation distance, given x and y calculate how similar the x and y is.
    return a value, 0 means x and y are independent, higher mean x are determinstic of y.
    """
    assert len(x)==len(y)
    n=len(x)
    a=np.zeros((n,n))
    b=np.zeros((n,n))
    ##calculate pair wise distance
    for i in xrange(n):
        for j in xrange(n):
            a[i,j]=np.linalg.norm(x[i]-x[j])
            b[i,j]=np.linalg.norm(y[i]-y[j])
    aRowMean=np.mean(a,axis=1)
    aColMean=np.mean(a,axis=0)
    bRowMean=np.mean(b,axis=1)
    bColMean=np.mean(b,axis=0)
    aMean=np.mean(a)
    bMean=np.mean(b)
    A=np.zeros((n,n))
    B=np.zeros((n,n))
    ##normalization
    for i in xrange(n):
        for j in xrange(n):
            A[i,j]=a[i,j]-aRowMean[i]-aColMean[j]+aMean
            B[i,j]=b[i,j]-bRowMean[i]-bColMean[j]+bMean
    cordis=0.
    for i in xrange(n):
        for j in xrange(n):
            cordis += A[i,j]*B[i,j]
    return cordis*(1./n**2)

In [5]:
def cosine_distance(u, v):
    return np.dot(u, v) / (math.sqrt(np.dot(u, u)) * math.sqrt(np.dot(v, v)))

In [6]:
#%matplotlib inline
from matplotlib import pyplot as plt
def plotFunction(x,y,text):
    n=x.shape[1]
    cols=2
    rows=int(math.ceil(n/2.))
    fig,ax=plt.subplots(ncols=cols,nrows=rows)
    if rows==1:
        for j in range(cols):
            if j<n:
                ax[j].scatter(x[:,j],y,c='b')
    else:
        for i in range(rows):
            for j in range(cols):
                if i*cols+j<n:
                    ax[i,j].scatter(x[:,i*cols+j],y,c='b')
                    #ax[i].title('x%d'%i,fontsize='small')
    plt.title(text)
    plt.show()


In [7]:
def crossValidation(x,y):
    """
    version 2
    according to the true y, predict
    greedy algorithm: first using index
    return
    """
    crossErrors=[]
    confusion_matrix=[]
    #svc=svm.LinearSVC()
    svc=svm.SVC(kernel='linear')
    kf = KFold(x.shape[0], n_folds=5,shuffle=True)
    for train,test in kf:
        svc.fit(x[train],y[train])
        prediction=svc.predict(x[test])
        crossErrors.append(((prediction!=y[test]).sum())/float(len(y[test])))
        confusion_matrix.append(CM(y[test],prediction))
    return np.mean(crossErrors),confusion_matrix

In [8]:
def layer_rbf(x,y):
    """version 2"""
    kpca=KernelPCA(kernel='rbf',gamma=10.,n_components=x.shape[1])
    xT=kpca.fit_transform(x)
    rank=[]
    numOfFeatures=xT.shape[1]
    for i in xrange(numOfFeatures):
        rank.append(corDis(xT[:,i],y))
    feaSelected=IndOfSortedFea(rank)
    
    featureSelectedErrors=[]
    for i in xrange(numOfFeatures):
        crossErrors,cms=crossValidation(xT[:,feaSelected[:i+1]],y)
        featureSelectedErrors.append(crossErrors)
    choice=featureSelectedErrors.index(min(featureSelectedErrors))
    return xT[:,feaSelected[:choice+1]],featureSelectedErrors,cms

In [9]:
def layer_poly(x,y):
    """ploy kernel"""
    kpca=KernelPCA(kernel='poly',n_components=x.shape[1])
    xT=kpca.fit_transform(x)
    rank=[]
    numOfFeatures=xT.shape[1]
    for i in xrange(numOfFeatures):
        rank.append(corDis(xT[:,i],y))
    feaSelected=IndOfSortedFea(rank)
    
    featureSelectedErrors=[]
    for i in xrange(numOfFeatures):
        crossErrors,cms=crossValidation(xT[:,feaSelected[:i+1]],y)
        featureSelectedErrors.append(crossErrors)
    choice=featureSelectedErrors.index(min(featureSelectedErrors))
    return xT[:,feaSelected[:choice+1]],featureSelectedErrors,cms

In [10]:
def layer_sigmoid(x,y):
    """sigmoid kernel"""
    kpca=KernelPCA(kernel='sigmoid',n_components=x.shape[1])
    xT=kpca.fit_transform(x)
    rank=[]
    numOfFeatures=xT.shape[1]
    for i in xrange(numOfFeatures):
        rank.append(corDis(xT[:,i],y))
    feaSelected=IndOfSortedFea(rank)
    
    featureSelectedErrors=[]
    for i in xrange(numOfFeatures):
        crossErrors,cms=crossValidation(xT[:,feaSelected[:i+1]],y)
        featureSelectedErrors.append(crossErrors)
    choice=featureSelectedErrors.index(min(featureSelectedErrors))
    return xT[:,feaSelected[:choice+1]],featureSelectedErrors,cms

In [11]:
def layer_cosine(x,y):
    kpca=KernelPCA(kernel='cosine',n_components=x.shape[1])
    xT=kpca.fit_transform(x)
    rank=[]
    numOfFeatures=xT.shape[1]
    for i in xrange(numOfFeatures):
        rank.append(corDis(xT[:,i],y))
    feaSelected=IndOfSortedFea(rank)
    
    featureSelectedErrors=[]
    for i in xrange(numOfFeatures):
        crossErrors,cms=crossValidation(xT[:,feaSelected[:i+1]],y)
        featureSelectedErrors.append(crossErrors)
    choice=featureSelectedErrors.index(min(featureSelectedErrors))
    return xT[:,feaSelected[:choice+1]],featureSelectedErrors,cms

In [12]:
def layer(x,y):
    """4*n features using different kernel"""
    kpca_cosine=KernelPCA(kernel='cosine',n_components=x.shape[1])
    kpca_sigmoid=KernelPCA(kernel='sigmoid',n_components=x.shape[1])
    kpca_poly=KernelPCA(kernel='poly',n_components=x.shape[1])
    kpca_rbf=KernelPCA(kernel='rbf',gamma=10.,n_components=x.shape[1])
    
    xT_cosine=kpca_cosine.fit_transform(x)
    xT_sigmoid=kpca_sigmoid.fit_transform(x)
    xT_poly=kpca_poly.fit_transform(x)
    xT_rbf=kpca_rbf.fit_transform(x)
    
    xT=np.concatenate((xT_cosine,xT_sigmoid,xT_poly,xT_rbf),axis=1)
    
    rank=[]
    numOfFeatures=xT.shape[1]
    for i in xrange(numOfFeatures):
        rank.append(cosine_distance(xT[:,i],y))
    feaSelected=IndOfSortedFea(rank)
    
    featureSelectedErrors=[]
    for i in xrange(numOfFeatures):
        crossErrors,cms=crossValidation(xT[:,feaSelected[:i+1]],y)
        featureSelectedErrors.append(crossErrors)
    choice=featureSelectedErrors.index(min(featureSelectedErrors))
    return xT[:,feaSelected[:choice+1]],featureSelectedErrors,cms

In [13]:
def pretrain(x,y):
    rank=[]
    numOfFeatures=x.shape[1]
    for i in xrange(numOfFeatures):
        rank.append(corDis(x[:,i],y))
    feaSelected=IndOfSortedFea(rank)
    
    featureSelectedErrors=[]
    for i in xrange(numOfFeatures):
        crossErrors,cms=crossValidation(x[:,feaSelected[:i+1]],y)
        featureSelectedErrors.append(crossErrors)
    choice=featureSelectedErrors.index(min(featureSelectedErrors))
    return x[:,feaSelected[:choice+1]],featureSelectedErrors,cms  

In [14]:
from matplotlib import pyplot as plt
def plotdata(x,y):
    n=x.shape[1]
    fig,ax=plt.subplots(ncols=n,nrows=n)
    fig.set_size_inches(9,6)
    for i in xrange(n):
        for j in xrange(i+1):
            for t,color in zip(xrange(3),'rgb'):
                ax[i][j].scatter(x[y == t,i],x[y == t,j],c=color)
    plt.show()
#plotdata(x,y)

In [15]:
def plotErr(err):
    x=range(len(err))
    #y=[e/min(err) for e in err]
    plt.bar(x,err)
    plt.title('Error Bar_Chart')
    plt.show()

In [16]:
def printCM(cm):
    """
    print cm in a nice way
    """
    for i in cm:
        print i

In [17]:
iris=load_iris()
x=iris.data
y=iris.target

In [18]:
#%matplotlib inline

In [19]:
result,error,cm=pretrain(x,y)

In [20]:
printCM(cm)

[[10  0  0]
 [ 0 12  0]
 [ 0  0  8]]
[[10  0  0]
 [ 0  6  1]
 [ 0  0 13]]
[[12  0  0]
 [ 0 10  0]
 [ 0  0  8]]
[[ 9  0  0]
 [ 0 12  0]
 [ 0  0  9]]
[[ 9  0  0]
 [ 0  9  0]
 [ 0  0 12]]


In [21]:
plotErr(error)

In [22]:
result1,e1,cm1=layer(result,y)
printCM(cm1)

[[ 8  0  0]
 [ 0  7  0]
 [ 0  1 14]]
[[12  0  0]
 [ 0  8  0]
 [ 0  0 10]]
[[13  0  0]
 [ 0  9  1]
 [ 0  0  7]]
[[ 9  0  0]
 [ 0 10  1]
 [ 0  1  9]]
[[ 8  0  0]
 [ 0 13  1]
 [ 0  1  7]]


In [23]:
plotErr(e1)

In [24]:
print result1.shape

(150L, 16L)


In [25]:
result2,e2,cm2=layer(result1,y)
printCM(cm2)

[[11  0  0]
 [ 0 10  0]
 [ 0  0  9]]
[[ 9  0  0]
 [ 0  7  0]
 [ 0  4 10]]
[[14  0  0]
 [ 0 10  1]
 [ 0  0  5]]
[[10  0  0]
 [ 0  9  0]
 [ 0  1 10]]
[[ 6  0  0]
 [ 0 11  2]
 [ 0  0 11]]


In [26]:
plotErr(e2)

In [27]:
print result2.shape

(150L, 11L)


In [ ]:
result3,e3,cm3=layer(result2,y)
printCM(cm3)

In [ ]:
plotErr(e3)

In [ ]:
result4,e4,cm4=layer(result3,y)
printCM(cm4)

In [ ]:
plotErr(e4)

In [ ]:
result5,e5,cm5=layer(result4,y)
printCM(cm5)

In [ ]:
plotErr(e5)

In [ ]:
result6,e6,cm6=layer(result5,y)
printCM(cm6)

In [ ]:
plotErr(e6)